In [21]:
# Full code for getting All_Articles, Top_Headlines and From that we will get (Keywords, Sentiment, Summary, 
#   Topic_Clasiification).

In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf
from newsapi import NewsApiClient
import nltk
import string
import re
from newspaper import Article
from keybert import KeyBERT
from textblob import TextBlob
import torch
import json
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [23]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sandeep\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
model = T5ForConditionalGeneration.from_pretrained('mrm8488/t5-base-finetuned-summarize-news')
tokenizer = T5Tokenizer.from_pretrained('mrm8488/t5-base-finetuned-summarize-news',return_dict=True)
device = torch.device('cpu')

In [25]:
main = tf.keras.models.load_model('text_classification_without_regions//')

In [26]:
d = {2:'Sports',3:'Politics',4:'Start-up',5:'Business',6:'Habitat',7:'Global Health',13:'Technology',14:'Travel',
     15:'Gender Equality',16:'Art & Culture',19:'Social Movement',
     20:'Local News',21:'Happy News',24:'Climate Crisis',25:'science',26:'Human Rights', 
     27:'Mental Health',28:'Biotech',29:'LGBT',
     30:'France',31:'Education',32:'Cryptocurrency',33:'Human Stories'}

In [27]:
from tkinter import *

In [28]:
from tkcalendar import Calendar
from tkcalendar import DateEntry
from datetime import datetime

In [29]:
newsapi = NewsApiClient(api_key='146cfc0db7b84967afa4fbb2d074ba21')

In [30]:
from nltk import tokenize

In [31]:
# Below code for Top_Headlines

In [21]:
root = Tk()
root.title('Results')
root.geometry('1400x800')

s = Label(root, text='Sources')
s.pack()

options = ['bbc-news','CNN','fox-news']

clicked = StringVar()
clicked.set(options[0])

drop = OptionMenu(root, clicked, *options)
drop.pack()

def get():
    top_headlines = newsapi.get_top_headlines(sources=clicked.get(),language='en')
    top_hlines.config(state='normal')
    top_hlines.delete('1.0', 'end')
    top_hlines.insert('1.0', top_headlines['articles'])
    top_hlines.config(state='disabled')

button = Button( root , text = "get" , command = get ).pack()
                    
tlabel = Label(root, text='Top Headlines')
tlabel.pack()
top_hlines = Text(root, height=15, width=150)
top_hlines.pack()


ulabel = Label(root, text='Please give the desired url')
ulabel.pack()

utext = Text(root, height=2,width=150)
utext.pack()

def kss():
    url = utext.get('1.0','end').strip()
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()
    text = article.text
    def clean_text(text):
      text = re.sub(r'[^a-zA-Z]', ' ', text)
      text = text.lower()
      text = text.split()
      text = [word for word in text if not word in set(nltk.corpus.stopwords.words('english'))]
      text = ' '.join(text)
      return text
    
    kb = KeyBERT('distilbert-base-nli-mean-tokens')
    keywords = kb.extract_keywords(text, stop_words='english')
    
    keys.config(state='normal')
    keys.delete('1.0', 'end')
    keys.insert('1.0', keywords)
    keys.config(state='disabled')
    
    analysis = TextBlob(text)
    a = analysis.polarity
    def type():
      if (a>0.2):
        return("Positive")
      elif (a<0):
        return("Negative")
      else:
        return("Neutral")
    c = type()
    
    sent.config(state='normal')
    sent.delete('1.0', 'end')
    sent.insert('1.0', c)
    sent.config(state='disabled')
    
    inputs = tokenizer.encode("summarize: " + text,return_tensors='pt',max_length=512,truncation=True)
    summary_ids = model.generate(inputs, max_length=120, min_length=80, length_penalty=5., num_beams=2)
    summary = tokenizer.decode(summary_ids[0])
    
    d = summary.split(" ")
    e = " ".join(d[1:])
    f = tokenize.sent_tokenize(e)
    g = f[:3]
    
    summ.config(state='normal')
    summ.delete('1.0', 'end')
    summ.insert('1.0', g)
    summ.config(state='disabled')
    
    k = [summary]
    l = main.predict(k)
    m = np.argsort(l)
    n = m.tolist()
    o = []
    for i in range(len(n[0])):
      last_item = n[0].pop()
      o.insert(i, last_item)
    p = o[:3]
    def top():
        for i in p:
            return(d[i])
    
    topic.config(state='normal')
    topic.delete('1.0', 'end')
    topic.insert('1.0', top())
    topic.config(state='disabled')
    
button = Button( root , text = "get" , command = kss ).pack()

klabel = Label(root, text='Keywords')
klabel.pack()

keys = Text(root, height=1, width=150)
keys.config(state='disabled', bg='#dddddd')
keys.pack()

xlabel = Label(root, text='Sentiment')
xlabel.pack()

sent = Text(root, height=1, width=150)
sent.config(state='disabled', bg='#dddddd')
sent.pack()

ylabel = Label(root, text='Summary')
ylabel.pack()

summ = Text(root, height=6, width=150)
summ.config(state='disabled', bg='#dddddd')
summ.pack()

zlabel = Label(root, text='Topic Classification')
zlabel.pack()

topic = Text(root, height=3, width=150)
topic.config(state='disabled', bg='#dddddd')
topic.pack()

root.mainloop()

In [22]:
# Below code for All_Articles

In [32]:
root = Tk()
root.title('Results')
root.geometry('1400x800')

#plabel for domains
p = Label(root, text='Domains')
p.pack()

opt = ['bbc.com','cnn.com','foxnews.com']

click = StringVar()
click.set(opt[0])

drop = OptionMenu(root, click, *opt)
drop.pack()

#calendar for selecting from date
cal_frame = Frame(root)
cal_frame.pack()

cal1=DateEntry(cal_frame, locale='en_US', date_pattern='mm/dd/y')
cal1.grid(row=1,column=0,padx=15)
st1 = datetime.strptime(str(cal1.get_date()), "%Y-%m-%d").strftime('%Y-%m-%d')

# calendar for selecting to date
cal2 = DateEntry(cal_frame, locale='en_US', date_pattern='mm/dd/y')
cal2.grid(row=1,column=1,padx=15)
st2 = datetime.strptime(str(cal2.get_date()), "%Y-%m-%d").strftime('%Y-%m-%d')

def art():
    all_articles = newsapi.get_everything(domains=click.get(),from_param=cal1.get_date(),to=cal2.get_date(),language='en',sort_by='relevancy',page=1)
    articles.config(state='normal')
    articles.delete('1.0', 'end')
    articles.insert('1.0', all_articles['articles'])
    articles.config(state='disabled')

button = Button( root , text = "get" , command = art).pack()

alabel = Label(root, text='All Articles')
alabel.pack()
articles = Text(root, height=10, width=150)
articles.pack()

ulabel = Label(root, text='Please give the desired url')
ulabel.pack()
utext = Text(root, height=2,width=150)
utext.pack()

def kss():
    url = utext.get('1.0','end').strip()
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()
    text = article.text
    def clean_text(text):
      text = re.sub(r'[^a-zA-Z]', ' ', text)
      text = text.lower()
      text = text.split()
      text = [word for word in text if not word in set(nltk.corpus.stopwords.words('english'))]
      text = ' '.join(text)
      return text
    
    kb = KeyBERT('distilbert-base-nli-mean-tokens')
    keywords = kb.extract_keywords(text, stop_words='english')
    
    keys.config(state='normal')
    keys.delete('1.0', 'end')
    keys.insert('1.0', keywords)
    keys.config(state='disabled')
    
    analysis = TextBlob(text)
    a = analysis.polarity
    def type():
      if (a>0.2):
        return("Positive")
      elif (a<0):
        return("Negative")
      else:
        return("Neutral")
    c = type()
    
    sent.config(state='normal')
    sent.delete('1.0', 'end')
    sent.insert('1.0', c)
    sent.config(state='disabled')
    
    inputs = tokenizer.encode("summarize: " + text,return_tensors='pt',max_length=512,truncation=True)
    summary_ids = model.generate(inputs, max_length=120, min_length=80, length_penalty=5., num_beams=2)
    summary = tokenizer.decode(summary_ids[0])
    
    d = summary.split(" ")
    e = " ".join(d[1:])
    f = tokenize.sent_tokenize(e)
    g = f[:3]
    
    summ.config(state='normal')
    summ.delete('1.0', 'end')
    summ.insert('1.0', g)
    summ.config(state='disabled')

    l = main.predict(g)
    m = np.argsort(l)
    n = m.tolist()
    o = []
    for i in range(len(n[0])):
      last_item = n[0].pop()
      o.insert(i, last_item)
    p = o[:3]
    def top():
        for i in p:
            return(d[i])
    
    topic.config(state='normal')
    topic.delete('1.0', 'end')
    topic.insert('1.0', top())
    topic.config(state='disabled')
    
button = Button( root , text = "get" , command = kss ).pack()

klabel = Label(root, text='Keywords')
klabel.pack()

keys = Text(root, height=2, width=150)
keys.config(state='disabled', bg='#dddddd')
keys.pack()

xlabel = Label(root, text='Sentiment')
xlabel.pack()

sent = Text(root, height=1, width=150)
sent.config(state='disabled', bg='#dddddd')
sent.pack()

ylabel = Label(root, text='Summary')
ylabel.pack()

summ = Text(root, height=6, width=150)
summ.config(state='disabled', bg='#dddddd')
summ.pack()

zlabel = Label(root, text='Topic Classification')
zlabel.pack()

topic = Text(root, height=2, width=150)
topic.config(state='disabled', bg='#dddddd')
topic.pack()


root.mainloop()